In [1]:
#-- 16 frames --
#-- Only 150 video for each class

In [2]:
#-- Install Libraries -------------------------------------------------------------------------------------------
!pip install pytorchvideo

from IPython import display
display.clear_output()
#-----------------------------------------------------------------------------------------------------------------

In [3]:
#-- Imports ------------------------------------------------------------------------------------------------------
import torch

from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 

import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split



import cv2

import numpy as np
import random

import os
import shutil
import copy
#-----------------------------------------------------------------------------------------------------------------

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [4]:
#-- Initialize ---------------------------------------------------------------------------------------------------
ds_input_path = '/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/'
ds_preprocessed_path = '/kaggle/working/ds/'
CLASS_NAMES = ['Violence', 'NonViolence']
#-----------------------------------------------------------------------------------------------------------------

In [5]:
#-- Create Folders  ----------------------------------------------------------------------------------------------
os.makedirs(ds_preprocessed_path, exist_ok=True)
#-----------------------------------------------------------------------------------------------------------------

In [6]:
#-- Check Number of Frames and Resolution of Videos ------------------------------------------------------------
i = 0
for root, dirs, files in os.walk(ds_input_path):
    
    for filename in files:
        file_path = os.path.join(root, filename)   
        
        if file_path.endswith(('.mp4')):          
            cap = cv2.VideoCapture(file_path)        

            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

            num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            print(f"Video: {filename} | number of frames: {num_frames} - Resolution: {width} x {height}")

            # Release the video capture object
            cap.release()
            
            i += 1
            
            if i>=10:
                break
#-------------------------------------------------------------------------------------------------------------

Video: NV_759.mp4 | number of frames: 125 - Resolution: 224 x 224
Video: NV_564.mp4 | number of frames: 125 - Resolution: 224 x 224
Video: NV_126.mp4 | number of frames: 150 - Resolution: 224 x 224
Video: NV_601.mp4 | number of frames: 105 - Resolution: 224 x 224
Video: NV_397.mp4 | number of frames: 55 - Resolution: 224 x 224
Video: NV_782.mp4 | number of frames: 115 - Resolution: 224 x 224
Video: NV_143.mp4 | number of frames: 150 - Resolution: 224 x 224
Video: NV_650.mp4 | number of frames: 145 - Resolution: 224 x 224
Video: NV_639.mp4 | number of frames: 145 - Resolution: 224 x 224
Video: NV_948.mp4 | number of frames: 145 - Resolution: 224 x 224
Video: V_465.mp4 | number of frames: 141 - Resolution: 1280 x 720


In [7]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4
num_clips = 10
num_crops = 3

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [8]:
#-- Function to Preprocess videos -------------------------------------------------------------------------------
def preprocess_video(video_path, output_path):   
    start_sec = 0
    end_sec = start_sec + clip_duration
    video = EncodedVideo.from_path(video_path)
    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
    video_data = transform(video_data)
    inputs = video_data["video"]  
    torch.save(inputs, output_path)  
#-------------------------------------------------------------------------------------------------------------

In [9]:
#-- Preprocees and Split Data ------------------------------------------------------------------------------------
def preprocess_all_videos(input_folder, out_dir):
    
    #-- Create output directories if they don't exist --    
    os.makedirs(out_dir, exist_ok=True)
    
    i=0

    for class_name in CLASS_NAMES:
        class_folder = os.path.join(input_folder, class_name)
       
        #-- Get all video files in the class folder --
        videos = [f for f in os.listdir(class_folder) if f.endswith(('.mp4', '.avi', '.mov'))]
        
        random.seed(42)
        random.shuffle(videos)
        
        videos = videos[0:150]      
        
        out_folder = os.path.join(ds_preprocessed_path, class_name)
        os.makedirs(out_folder, exist_ok=True)
            
        
        for video in videos:
            i = i+1
            video_path = os.path.join(class_folder, video)            
            output_path = os.path.join(out_folder, video.replace('.mp4', '.pth'))
            preprocess_video(video_path, output_path)
            print(f'i= {i} :Processed and saved {video} to {out_folder}')
#-------------------------------------------------------------------------------------------------------------

In [10]:
#-- Run the preprocessing and splitting --
preprocess_all_videos(input_folder= ds_input_path,
                      out_dir= ds_preprocessed_path)
#-------------------------------------------------------------------------------------------------------------

i= 1 :Processed and saved V_39.mp4 to /kaggle/working/ds/Violence
i= 2 :Processed and saved V_126.mp4 to /kaggle/working/ds/Violence
i= 3 :Processed and saved V_758.mp4 to /kaggle/working/ds/Violence
i= 4 :Processed and saved V_558.mp4 to /kaggle/working/ds/Violence
i= 5 :Processed and saved V_700.mp4 to /kaggle/working/ds/Violence
i= 6 :Processed and saved V_253.mp4 to /kaggle/working/ds/Violence
i= 7 :Processed and saved V_372.mp4 to /kaggle/working/ds/Violence
i= 8 :Processed and saved V_278.mp4 to /kaggle/working/ds/Violence
i= 9 :Processed and saved V_498.mp4 to /kaggle/working/ds/Violence
i= 10 :Processed and saved V_698.mp4 to /kaggle/working/ds/Violence
i= 11 :Processed and saved V_705.mp4 to /kaggle/working/ds/Violence
i= 12 :Processed and saved V_33.mp4 to /kaggle/working/ds/Violence
i= 13 :Processed and saved V_528.mp4 to /kaggle/working/ds/Violence
i= 14 :Processed and saved V_94.mp4 to /kaggle/working/ds/Violence
i= 15 :Processed and saved V_287.mp4 to /kaggle/working/ds/V

In [11]:
#-- Zip Data -----------------------------------------------------------------------------------------------------
ds_zip_file = '/kaggle/working/data'
shutil.make_archive(ds_zip_file, 'zip', ds_preprocessed_path)
shutil.rmtree(ds_preprocessed_path)
#-------------------------------------------------------------------------------------------------------------